In [84]:
import pennylane as qml
from pennylane.fourier import coefficients, _
from functools import partial

from itertools import product
from time import time
import plotly.express as px
import numpy as np

In [85]:
rng = np.random.default_rng(1111)

n_qubits = 2
n_layers = 1
max_freq = n_qubits * n_layers

dev = qml.device("default.qubit", wires=n_qubits)

weights = rng.random(size=(n_layers*n_qubits*2))
x = 0.1

In [86]:

@qml.qnode(dev)
def circuit_with_weights(w):
    w_idx = 0
    for l in range(n_layers):
        for q in range(n_qubits):
            qml.RY(w[w_idx], wires=q)
            w_idx += 1
            qml.RZ(w[w_idx], wires=q)
            w_idx += 1

        if n_qubits > 1:
            for q in range(n_qubits):
                qml.CNOT(wires=[q, (q + 1) % n_qubits])

    return qml.expval(qml.PauliZ(0))


In [87]:
def fourier_series(coeffs, frequencies, x):
    fs_sum = 0
    for c, omega in zip(coeffs, frequencies):
        # print(f"c_{omega} = (c_{-omega})* = {c}")
        fs_sum += c * np.exp(1j * x * omega)
        if omega > 0:
            fs_sum += np.conjugate(c) * np.exp(1j * x * -omega)
    return fs_sum

In [88]:
def recursive_fourier_series(coeffs, degree, xvec):
    n_ranges = [np.arange(-d, d + 1) for d in np.array((degree,)*len(xvec))]
    nvecs = product(*(n_ranges))

    mfs_sum = 0
    for c, nvec in zip(coeffs.flatten(), nvecs):
        mfs_prod = 1
        for x, n in zip(xvec, nvec):
            mfs_prod *= c * np.exp(1j * x * n)
            if n > 0:
                mfs_prod += np.conjugate(c) * np.exp(1j * x * -n)
        mfs_sum += mfs_prod

    return mfs_sum

In [ ]:
def coefficients(f, degree):
    if isinstance(max_freq, int):
        max_freq = [max_freq]

    degree = np.array(max_freq)

    # number of integer values for the indices n_i = -degree_i,...,0,...,degree_i
    k = 2 * degree + 1

    # create generator for indices nvec = (n1, ..., nN), ranging from (-d1,...,-dN) to (d1,...,dN)
    n_ranges = [np.arange(d + 1) for d in degree]
    nvecs = product(*(n_ranges))

    # here we will collect the discretized values of function f
    f_discrete = np.zeros(shape=tuple(k))

    spacing = 2 * np.pi / k

    for nvec in nvecs:
        sampling_point = spacing * np.array(nvec)
        # fill discretized function array with value of f at inpts
        f_discrete[nvec] = f(sampling_point)

    coeffs = np.fft.fftn(f_discrete) / k

    return coeffs

In [89]:

# freqs = np.concatenate([np.arange(max_freq+1), np.arange(-max_freq, 0)])
coeffs = coefficients(circuit_with_weights, len(weights), max_freq)

print(coeffs.shape)
print(coeffs)
start = time()
res_fft = recursive_fourier_series(coeffs, max_freq, weights)
end = time()
print(f"Calculation using FFT took {end-start}")
print(f"Result using FFT: {np.real(res_fft)}")

start = time()
res_pl = circuit_with_weights(weights)
end = time()
print(f"Calculation using pennylane took {end-start}")
print(f"Result using Pennylane: {res_pl}")


(5, 5, 5, 5)
[[[[ 3.97903932e-17+0.j  2.13162821e-18+0.j -1.42108547e-17+0.j
    -1.42108547e-17+0.j  2.13162821e-18+0.j]
   [ 5.00000000e-01+0.j  5.25132043e-18+0.j  1.54877836e-18+0.j
     1.54877836e-18+0.j  5.25132043e-18+0.j]
   [-2.55795385e-17+0.j  1.67647125e-18+0.j  3.33620295e-18+0.j
     3.33620295e-18+0.j  1.67647125e-18+0.j]
   [-2.55795385e-17+0.j  1.67647125e-18+0.j  3.33620295e-18+0.j
     3.33620295e-18+0.j  1.67647125e-18+0.j]
   [ 5.00000000e-01+0.j  5.25132043e-18+0.j  1.54877836e-18+0.j
     1.54877836e-18+0.j  5.25132043e-18+0.j]]

  [[-2.84217094e-18+0.j -4.71463218e-18+0.j  2.53495177e-18+0.j
     1.92807583e-18+0.j -2.14386441e-18+0.j]
   [ 1.13686838e-17+0.j -4.38433221e-18+0.j -7.88957792e-18+0.j
     3.82375578e-18+0.j  1.06114804e-17+0.j]
   [ 1.11613502e-17+0.j  3.03640825e-18+0.j -2.28735154e-18+0.j
    -3.19232219e-19+0.j -2.70093548e-18+0.j]
   [ 1.11613502e-17+0.j  3.03640825e-18+0.j -2.28735154e-18+0.j
    -3.19232219e-19+0.j -2.70093548e-18+0.j]
   [